### CMPS242 Homework 5 - Classifying Donald Trump and Hillary Clinton's Tweets
#### Team Members:
Alex                       
Karthik Balakrishna           
Roy Shadmon     
Sasidharan Mahalingam


#### Importing the required packages
First import the rerquired packages required to create a neural network

In [1]:
#Importing Packages
import tensorflow as tf
import os
import pandas as pd
import numpy as np
#Clearing any previous graph if present
tf.reset_default_graph()

#### Loading Train data
Import the tweets from the given dataset

In [2]:
#TODO: Have to implement the text manupilation to remove the https and other unwanted characters and strings

#Loading the train dataset
df1 = pd.read_csv('HillaryTrumpTrainWithTags.csv')
#Loading the test dataset
df2 = pd.read_csv('TrumpHillaryTestWithTags.csv')
#Getting only the tweets from the train dataset
train_tweets = df1['tweet'].tolist()
#Getting only the tweets from the test dataset
test_tweets = df2['tweet'].tolist()
#Printing a sample of the train dataset tweets
print('Printing a sample of the train dataset tweets... \n\n')
print(train_tweets[0:5])
#Printing a sample of the test dataset tweets
print('Printing a sample of the test dataset tweets... \n\n')
print(test_tweets[0:5])
#Concatenating the train and test dataset tweets to form the word embeddings
tweets = train_tweets + test_tweets

Printing a sample of the train dataset tweets... 


['In Tampa, Florida- thank you to all of our outstanding volunteers who want to <hashtag> MakeAmericaGreatAgain! <url>', 'Poll: <user>  realDonaldTrump vs. <user>  HillaryClinton among white Evangelicals. <url>', 'Obama on whether Trump could be trusted with US nuclear weapons: "Make your own judgment" <url>', '"Hillary Clinton has never quit on anything in her life." — <user>  FLOTUS <hashtag> DemsInPhilly <url>', 'I LOVE NEW YORK! <hashtag> NewYorkValues\n<url>']
Printing a sample of the test dataset tweets... 


['Live from Charlotte: <user> POTUS hits the trail with Hillary for the first time in this campaign. Watch: <url>', 'The worst part is, Trump\'s disturbing policies for immigrant families go way beyond just "building a wall." <url>', 'You can watch 360 video live from the podium! <url>', 'No wonder Donald Trump is hiding his tax returns. <hashtag> debatenight <url>', '"Let us be vigilant, but not afraid…we choose resolve, n

#### Tokenizing the tweets
Tokenize the tweets using keras Tokenizer

In [3]:
#Tokenize all the tweets
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(tweets)
#Printing some of the word indexs
sample_mapping = {k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]} 
print('Printing a few word index mappings... \n\n')
print(sample_mapping)

word_dict = tokenizer.fit_on_texts(tweets)
print(word_dict)

Printing a few word index mappings... 


{'the': 1, 'url': 2, 'user': 3, 'to': 4, 'a': 5, 'hashtag': 6, 'and': 7, 'in': 8, 'of': 9, 'is': 10}
None


#### Loading GloVe word embeddings
Load the downloaded pre-trained GloVe word vectors

In [4]:
#Create an empty dictionary that stores the word to index mappings
embeddings_index = {}
#Import the GloVe word vectors
glove_data = 'glove.twitter.27B.200d.txt'
#Load the word vectors into memory
f = open(glove_data)
for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = value
f.close()

#Printing the number of word vectors loaded
print('Loaded %s word vectors.' % len(embeddings_index))


FileNotFoundError: [Errno 2] No such file or directory: 'glove.twitter.27B.200d.txt'

#### Mapping the words to the GloVe word vectors
Map the tokenized words to 200 dimensional word vectors

In [ ]:
#Setting the dimension of the word vectors

#Note: I have used 200 dimensional vectors (I know it might overfit as the given data is very less). 
#Let's zero down on the optimal value after we get this to work

embedding_dimension = 20
#Getting the word index representation of the tokenized words
word_index = tokenizer.word_index


embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimension), dtype=np.float64)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector[:embedding_dimension]


#Printing the shape of the word embedding matrix
print(embedding_matrix.shape)
print(type(embedding_matrix))
#TODO:    
#One future improvement that we could do is to train these embedding using the tweets instead of setting the words not present to zeros

#### Get the tweets from the training dataset and find word embedding matrix
Get the training tweet dataset and convert that from strings into the word embedding matrix, the required input to the LSTM

In [ ]:
#reshaping the training data to word to index sequences
train_X = tokenizer.texts_to_sequences(train_tweets)
#pad the shorter sentences to the length of the largest sentence
train_X = tf.keras.preprocessing.sequence.pad_sequences(train_X)
#print the shape of the matrix of word vectors
print(train_X.shape)
print(type(train_X))

#### Get the tweets from the test dataset and find the word embedding matrix for it
Get the test tweets and find the word embedding for the tweets

In [ ]:
#reshaping the test data to word to index sequences
test_X = tokenizer.texts_to_sequences(test_tweets)
#pad the shorter sentences to the length of the largest sentence
test_X = tf.keras.preprocessing.sequence.pad_sequences(test_X, maxlen=32)
#print the shape of the matrix of word vectors
print(test_X.shape)
print(type(test_X))

#### Define different parameters for the LSTM
Now we define the number of time steps, number of hidden layers and the number of nodes in each layer

In [ ]:
#Set number of time steps
time_steps = 32
#Batch size
batch_size = 3000
#Number of hidden layers
num_units = 10
#Learning rate
learning_rate = 0.0001
#Number of classes
n_classes = 2
#Size of the word vectors
n_input = embedding_dimension
#Regularizer parameter
lambda_l1 = 0.001

#### Creating labels for train and test data set
Next, we form the train and test labels from the loaded dataframe

In [ ]:
#Creating an empty list to store the labels
l1 = []
#Get the train labels
_ = df1['handle'].apply(lambda x: l1.append([1,0]) if x == 'realDonaldTrump' else l1.append([0,1]))
#Convert the train label list into an numpy array
train_labels = np.array(l1, dtype=np.float64)
print(train_labels.shape)
#Empty the list to store the test labels
l1 = []
#Populate the list with test labels
_ = df2['handle'].apply(lambda x: l1.append([1,0]) if x == 'realDonaldTrump' else l1.append([0,1]))
#Convert the test label list into an numpy array
test_labels = np.array(l1, dtype=np.float64)
print(test_labels.shape)

#### Declaring the weeights and biases for the final fed forward network and placeholders for the input data
Declaring variables for the weights and biases for the final feed forward network and the placeholders for the input dataset

In [ ]:
#Creating a variable to hold the weights of the final feed forward layer
out_weights = tf.Variable(tf.random_normal([num_units, n_classes], dtype=tf.float64), dtype=tf.float64)
#Creating a variable to hold the biases of the final feed forward layer
out_bias = tf.Variable(tf.random_normal([n_classes],dtype=tf.float64),dtype=tf.float64)

#Input data placeholder
x = tf.placeholder("float64",[None,time_steps,n_input])
#x = tf.identity(embed)
#Input label placeholder
y = tf.placeholder("float64",[None,n_classes])
#y = tf.identity(train_labels)
X = tf.placeholder("int64", [None,32])
word_vecs = tf.placeholder("float64",[14120, n_input])


#### Defining the LSTM model
Then, define and construct the LSTM model

In [ ]:

#Copying values into placeholders
x = tf.nn.embedding_lookup(word_vecs, X)

#Reshaping the input dataset
input_data = tf.unstack(x, time_steps, 1)

#defining the network
lstm_layer= tf.contrib.rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_= tf.contrib.rnn.static_rnn(lstm_layer,input_data,dtype="float64")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#defining regularization loss
reg_loss = lambda_l1 * sum( tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
loss += reg_loss
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

#### Running the LSTM model
Now that we have set up the LSTM model, we run the LSTM model

In [ ]:
def generate_samples(in_data, in_labels, no_of_samples):
    indices = np.random.choice(in_data.shape[0],no_of_samples,replace=False)
    return(in_data[indices], in_labels[indices])


#initialize variables
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<10000:
        batch_x, batch_y = generate_samples(train_X, train_labels, batch_size)
        #print(batch_x.shape)
        #batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={word_vecs:embedding_matrix, X: batch_x, y: batch_y})
        
        if iter %1000 == 0:
            acc = sess.run(accuracy,feed_dict={word_vecs:embedding_matrix, X: batch_x, y: batch_y})
            los = sess.run(loss,feed_dict={word_vecs:embedding_matrix, X: batch_x, y: batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
    output = sess.run(tf.argmax(prediction, 1), feed_dict={word_vecs:embedding_matrix, X: test_X, y: test_labels})
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={word_vecs:embedding_matrix, X: test_X, y: test_labels}))
    print("your prediction for X_test is :", output)